In [ ]:
import sys
sys.path.append("../")

from ExtractFeatures import extract_features
from PreProcessing import PreProcessing

from GaborFilter.GaborFilter import generate_gabor_kernel_cv, gabor_filter
import skimage as ski
import tqdm
import pandas as pd
import os
import numpy as np

In [ ]:
import skimage as ski
import numpy as np
kernels = []
thetas = [0 , np.pi/4 , np.pi/2 , 3*np.pi/4] # mentioned in the papers  0  , 45 , 90 , 135
for theta in thetas : 
    for frequency in [0.1, 0.2, 0.3, 0.4] : 
        kernel = ski.filters.gabor_kernel(frequency=frequency , theta = theta )
        kernels.append(np.real(kernel))
# Convert kernels to real values
# kernels = [np.real(kernel) for kernel in kernels]

img = ski.io.imread("../data/processed/fonts-dataset/Scheherazade New/4.jpeg")
features = extract_features(img,kernels)
# print(features)
print(len(features))

In [ ]:
Folders = ['IBM Plex Sans Arabic', 'Lemonada', 'Marhey', 'Scheherazade New']

columns=['Area Ratio']
for i in range(1, 17):
    columns.append('gabor mean ' + str(i))
    columns.append('gabor std ' + str(i))
    columns.append('gabor energy ' + str(i))

for i in range(1, 6):
    columns.append('glcm ' + str(i))
for i in range(1, 27):
    columns.append('lbp ' + str(i))
# for i in range(1, 26):
#     columns.append('zernike ' + str(i))

columns.append('font_type')

features = pd.DataFrame(columns=columns)
# kernels = generate_gabor_kernel_cv()
import skimage as ski
import numpy as np
kernels = []
thetas = [0 , np.pi/4 , np.pi/2 , 3*np.pi/4] # mentioned in the papers  0  , 45 , 90 , 135
for theta in thetas : 
    for frequency in [0.05, 0.01, 0.15, 0.3] : 
        kernel = ski.filters.gabor_kernel(frequency=frequency , theta = theta )
        kernels.append(np.real(kernel))

import random
# random_numbers = random.sample(range(1000), 1000)

# for i in tqdm.tqdm(random_numbers):
for i in tqdm.tqdm(range(1000)):
    for j in range(4):
        # pathname = "../data/processed_wo_hough/fonts-dataset/" + Folders[j] + "/" + str(i) + ".jpeg"
        # pathname = "../data/processed/fonts-dataset/" + Folders[j] + "/" + str(i) + ".jpeg"
        pathname = "../data/raw/fonts-dataset/" + Folders[j] + "/" + str(i) + ".jpeg"
        if not os.path.exists(pathname):
            continue
        img = ski.io.imread(pathname)
        if len(np.unique(img)) == 1 : 
                continue # all image contain the same color 
        preprocessed_img = PreProcessing.PreProcessing(img)
        features_vect = extract_features(preprocessed_img, kernels)
        # print(features_vect)
        features_vect = np.append(features_vect, j)
        features = pd.concat([features, pd.DataFrame([features_vect], columns=columns)])

features.to_csv("./features_1000_again.csv")


In [ ]:
from sklearn.preprocessing import StandardScaler

import sys
sys.path.append("../")
from preprocessing.PreProcessing import PreProcessing
kernels = []
thetas = [0 , np.pi/4 , np.pi/2 , 3*np.pi/4] # mentioned in the papers  0  , 45 , 90 , 135
for theta in thetas : 
    for frequency in [0.05, 0.01, 0.15, 0.3] : 
        kernel = ski.filters.gabor_kernel(frequency=frequency , theta = theta )
        kernels.append(np.real(kernel))

dataset = pd.read_csv('./features_1000_glcm.csv')    
# dataset = pd.read_csv('../../features/features_1000_wo_heggi.csv')    
# column split
x_data = dataset.drop(columns=['Unnamed: 0'])     
# Check the number of unique values in each column
unique_counts = dataset.nunique()

# Get the column names where the number of unique values is 1
columns_to_drop = unique_counts[unique_counts == 1].index

# Drop the columns
x_data = dataset.drop(columns=columns_to_drop)
# print(dataset.columns)
x_data = x_data.iloc[:, :-1]    
# print("x_data" , x_data)                               
scaler = StandardScaler()
print(x_data.shape)
scaler.fit(x_data)

print("scaler mean : , " ,scaler.mean_ )
print("scaler variance : , " ,scaler.var_ )
# //////////////////////
pathname = "../data/raw/fonts-dataset/Lemonada/7.jpeg"
img = ski.io.imread(pathname)
img = PreProcessing(img)
print("image" , img )

features_vect = extract_features(img, kernels)
print("feature_vect before reshape : ", features_vect)
print ("feature vector shape" ,features_vect.shape )
print("feature vector shape " , type(features_vect))
features = features.reshape(1,-1)
print("feature_vect after reshape:", features_vect)
features_vect = scaler.transform(features)
